# Huggingface SageMaker-SDK - GPT2 Fine-tuning example

1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
    1. [Installation](#Installation)  
    2. [Permissions](#Permissions)
    3. [Uploading data to sagemaker_session_bucket](#Uploading-data-to-sagemaker_session_bucket)  
3. [Fine-tuning & starting Sagemaker Training Job](#Fine-tuning-\&-starting-Sagemaker-Training-Job)  
    1. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
    2. [Download fine-tuned model from s3](#Download-fine-tuned-model-from-s3)
    3. [Text Generation on Local](#Text-Generation-on-Local)  

# Introduction

このnotebookはHuggingFaceの[run_clm.py](https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm.py)を日本語データで動作する様に変更を加えたものです。    

**日本語データで動作する様に変更を加えた以外はSageMakerで実行するために変更を加えた部分はありません**

データは[wikiHow日本語要約データセット](https://github.com/Katsumata420/wikihow_japanese)を使用します。    

このデモでは、AmazonSageMakerのHuggingFace Estimatorを使用してSageMakerのトレーニングジョブを実行します。    

_**NOTE: このデモは、SagemakerNotebookインスタンスで動作検証しています**_    

# Development Environment and Permissions

## Installation

このNotebookはSageMakerの`conda_pytorch_p36`カーネルを利用しています。    

**_Note: このnotebook上で推論テストを行う場合、（バージョンが古い場合は）pytorchのバージョンアップが必要になります。_**

In [ ]:
!pip install --upgrade pip
!pip install --upgrade torch
!pip install "sagemaker>=2.48.1" "transformers==4.9.2" "datasets[s3]==1.11.0" --upgrade
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")

## Permissions

ローカル環境でSagemakerを使用する場合はSagemakerに必要な権限を持つIAMロールにアクセスする必要があります。[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください

In [ ]:
import sagemaker


sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# データの準備

事前に`create_wikihow_dataset.ipynb`を実行してwikiHow日本語要約データセットを用意してください。

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
train = pd.read_json('./wikihow_japanese/data/output/train.jsonl', orient='records', lines=True)
train

In [ ]:
dev = pd.read_json('./wikihow_japanese/data/output/dev.jsonl', orient='records', lines=True)
dev

In [ ]:
with open('train.txt', 'w') as output_file:
    for row in tqdm(train.itertuples(), total=train.shape[0]):
        src = row.src
        tgt = row.tgt

        tokens = tokenizer.tokenize(src)
        src = "".join(tokens).replace('▁', '')
        text = '<s>' + src + '[SEP]' + tgt + '</s>'
        output_file.write(text + '\n')

In [ ]:
with open('dev.txt', 'w') as output_file:
    for row in tqdm(dev.itertuples(), total=dev.shape[0]):
        src = row.src
        tgt = row.tgt

        tokens = tokenizer.tokenize(src)
        src = "".join(tokens).replace('▁', '')
        text = '<s>' + src + '[SEP]' + tgt + '</s>'
        output_file.write(text + '\n')

## Uploading data to `sagemaker_session_bucket`

S3へデータをアップロードします。

In [ ]:
s3_prefix = 'samples/datasets/wikihow'

input_train = sess.upload_data(
    path='train.txt', 
    key_prefix=f'{s3_prefix}/train'
)

input_validation = sess.upload_data(
    path='dev.txt', 
    key_prefix=f'{s3_prefix}/valid'
)

In [ ]:
# データのUpload path

print(input_train)
print(input_validation)

# Fine-tuning & starting Sagemaker Training Job

`HuggingFace`のトレーニングジョブを作成するためには`HuggingFace` Estimatorが必要になります。    
Estimatorは、エンドツーエンドのAmazonSageMakerトレーニングおよびデプロイタスクを処理します。 Estimatorで、どのFine-tuningスクリプトを`entry_point`として使用するか、どの`instance_type`を使用するか、どの`hyperparameters`を渡すかなどを定義します。


```python
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    base_job_name='huggingface-sdk-extension',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    role=role,
    hyperparameters={
        'epochs': 1,
        'train_batch_size': 32,
        'model_name':'distilbert-base-uncased'
    }
)
```

SageMakerトレーニングジョブを作成すると、SageMakerは`huggingface`コンテナを実行するために必要なec2インスタンスの起動と管理を行います。    
Fine-tuningスクリプト`train.py`をアップロードし、`sagemaker_session_bucket`からコンテナ内の`/opt/ml/input/data`にデータをダウンロードして、トレーニングジョブを実行します。


```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
```

`HuggingFace estimator`で定義した`hyperparameters`は、名前付き引数として渡されます。

またSagemakerは、次のようなさまざまな環境変数を通じて、トレーニング環境に関する有用なプロパティを提供しています。

* `SM_MODEL_DIR`：トレーニングジョブがモデルアーティファクトを書き込むパスを表す文字列。トレーニング後、このディレクトリのアーティファクトはモデルホスティングのためにS3にアップロードされます。

* `SM_NUM_GPUS`：ホストで使用可能なGPUの数を表す整数。

* `SM_CHANNEL_XXXX`：指定されたチャネルの入力データを含むディレクトリへのパスを表す文字列。たとえば、HuggingFace estimatorのfit呼び出しで`train`と`test`という名前の2つの入力チャネルを指定すると、環境変数`SM_CHANNEL_TRAIN`と`SM_CHANNEL_TEST`が設定されます。

このトレーニングジョブをローカル環境で実行するには、`instance_type='local'`、GPUの場合は`instance_type='local_gpu'`で定義できます（GPUの場合は追加で設定が必要になります[SageMakerのドキュメント](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)を参照してください）。    
**_Note：これはSageMaker Studio内では機能しません_**

In [ ]:
# requirements.txtはトレーニングジョブの実行前に実行されます（コンテナにライブラリを追加する際に使用します）
# ファイルはここを参照しています。https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/requirements.txt
# 1点異なる部分は transformers >= 4.8.0 でHuggingFaceコンテナのバージョンが古く本家に追いついていないため、バージョンアップを行なっています。
!pygmentize ./scripts/requirements.txt

In [ ]:
# トレーニングジョブで実行されるコード
!pygmentize ./scripts/run_clm.py

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'model_name_or_path':'rinna/japanese-gpt2-medium',
    'train_file': '/opt/ml/input/data/train/train.txt',
    'validation_file': '/opt/ml/input/data/validation/dev.txt',
    'do_train': 'True',
    'do_eval': 'True',
    'num_train_epochs': 10,
    'per_device_train_batch_size': 1,
    'per_device_eval_batch_size': 1,
    'use_fast_tokenizer': 'False',
    'save_steps': 1000,
    'save_total_limit': 1,
    'output_dir':'/opt/ml/model',
}

## Creating an Estimator and start a training job

In [ ]:
# estimator
huggingface_estimator = HuggingFace(
    role=role,
    entry_point='run_clm.py',
    source_dir='./scripts',
    instance_type='ml.p3.8xlarge',
    instance_count=1,
    volume_size=200,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
    hyperparameters=hyperparameters,
)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': input_train, 'validation': input_validation})

# ml.p3.8xlarge, 10 epochでの実行時間の目安
# Training seconds: 3623
# Billable seconds: 3623

## Download-fine-tuned-model-from-s3


In [ ]:
import os

OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
from sagemaker.s3 import S3Downloader

# 学習したモデルのダウンロード
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path='.', # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

In [ ]:
# OUTPUT_DIRに解凍します

!tar -zxvf model.tar.gz -C output

## Text Generation on Local

In [ ]:
import torch
from transformers import AutoModelForCausalLM, T5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained('output/')
model.to(device)
model.eval()

In [ ]:
def generate_summary(body, num_gen=5):
    input_text = '<s>'+body+'[SEP]'
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, top_p=0.95, top_k=40, 
                         num_return_sequences=num_gen, max_length=1024, bad_words_ids=[[1], [5]])
    print('='*5,'原文', '='*5)
    print(body)
    print('-'*5, '要約', '-'*5)
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('</s>')[1]
        sent = sent.replace('</s>', '')
        print(sent)

In [ ]:
body = dev.src[0]
generate_summary(body)

In [ ]:
body = dev.src[1]
generate_summary(body)

In [ ]:
body = dev.src[2]
generate_summary(body)